# Imports and Custom Modules

This section of the notebook imports essential libraries and custom modules required for data analysis and visualization:

- **Standard Libraries**:
    - `sys`: Provides access to system-specific parameters and functions.
    - `pandas`: Used for data manipulation and analysis.
    - `random`: Generates random numbers for simulation purposes.
    - `datetime`: Handles date and time operations.

- **Custom Modules**:
    - `generate_signals`: A custom module for generating trading signals based on stock data.
    - `generate_candlestick_df`: A custom module for formatting data for candlestick chart generation.

In [216]:
!pip install polygon-api-client
import sys
import pandas as pd
import random
from datetime import datetime, timedelta
from generate_signals import generate_signals  # Importing signal generation function

# Setup and Customization Variables

This section provides a brief description of the key variables used for configuring and running the program:

1. **Timeframe and Date Variables**:
    - `start_date` and `end_date`: Define the simulation's date range.
    - `random_start_date` and `random_dates`: Specify random start dates for the simulation.
    
2. **Market Session Variables**:
    - `market_open`, `market_close`, and `pre_market_start`: Define the pre-market, open, and close times.

3. **Simulation Parameters**:
    - `number_of_days`: Number of days to plot and retrieve data for.
    - `intervalAmt`: Time interval for candlestick data aggregation.

In [217]:
# Generate a random start date within the range
start_date = datetime(2023, 5, 1)
end_date = datetime(2025, 3, 31)
number_of_days = random.randint(2, 7)
intervalAmt = 15
# Define market session times
market_open = '09:30:00'
market_close = '16:00:00'
pre_market_start = '04:00:00'

# Calculate the maximum possible start date to ensure the range fits `number_of_days`
max_start_date = end_date - timedelta(days=number_of_days - 1)
random_start_date = start_date + timedelta(days=random.randint(0, (max_start_date - start_date).days))

# Generate continuous dates
random_dates = [random_start_date + timedelta(days=i) for i in range(number_of_days)]

# Data Reformatting and Indicator Generation

The cell below is designed to process and analyze stock market data for visualization and trading signal generation. It performs the following key functions:

1. **Data Filtering and Transformation**:
  - Filters a large dataset (`df`) to extract relevant rows based on specific dates (`formatted_random_dates`).
  - Converts and reformats columns (e.g., `Datetime` to `Timestamp`) for consistency and ease of analysis.

2. **Technical Indicator Calculation**:
  - Computes key indicators like:
    - **8EMA**: Exponential Moving Average over an 8-period timeframe.
    - **VWAP**: Volume Weighted Average Price.
  - Adds these indicators as new columns to the filtered dataset (`retrieved_data`).

3. **Session and Day Segmentation**:
  - Segments data into distinct trading sessions (e.g., pre-market, regular market) using timestamps.
  - Maps each row to a specific trading day using a `day_mapping` dictionary.

4. **Key Levels Calculation**:
  - Calculates critical price levels for each trading day:
    - **ORB High/Low**: Opening Range Breakout levels (first 15 minutes of market open).
    - **PM High/Low**: Pre-market high and low prices.
    - **Yesterday's High/Low**: Previous day's high and low prices.
  - Stores these levels in lists (`orb_highs`, `orb_lows`, `pm_highs`, `pm_lows`, `yest_highs`, `yest_lows`) and maps them back to the dataset.

5. **Signal Generation**:
  - Applies a custom signal generation function (`generate_signals`) to identify trading opportunities (e.g., "BUY CALL", "BUY PUT") and calculate stop-loss levels.

### Key Variables:
- **`retrieved_data`**: The main DataFrame containing filtered and processed stock data with added indicators, session information, and trading signals.
- **`orb_highs`, `orb_lows`, `pm_highs`, `pm_lows`, `yest_highs`, `yest_lows`**: Lists storing calculated price levels for each trading day.
- **`day_mapping`**: A dictionary mapping unique dates to sequential day indices.
- **`formatted_random_dates`**: A list of dates used to filter the dataset.
- **`generate_signals`**: A custom function applied to generate trading signals and stop-loss levels.

This cell prepares the data for further analysis, visualization, and decision-making in the context of stock trading. It ensures the data is enriched with technical indicators and key levels, making it suitable for candlestick chart generation and trading strategy evaluation.

In [218]:
# Load the CSV file into a DataFrame and capitalize the first letter of column titles
df = pd.read_csv('df_2022_2024.csv')
df.columns = [col.capitalize() for col in df.columns]

# Convert the 'datetime' column to string
df['Datetime'] = df['Datetime'].astype(str)

# Reformat random_dates to match the format in the 'datetime' column
formatted_random_dates = [date.strftime('%Y-%m-%d') for date in random_dates]

# Filter the dataframe to find rows where the 'datetime' column starts with any of the formatted_random_dates
retrieved_data = df[df['Datetime'].str.startswith(tuple(formatted_random_dates))]

# Calculate 8EMA (Exponential Moving Average over an 8-period timeframe)
retrieved_data['8EMA'] = retrieved_data['Close'].ewm(span=8, adjust=False).mean()

# Calculate VWAP (Volume Weighted Average Price)
retrieved_data['VWAP'] = (retrieved_data['Close'] * retrieved_data['Volume']).cumsum() / retrieved_data['Volume'].cumsum()

# Convert 'Datetime' column to datetime type for filtering
retrieved_data['Datetime'] = pd.to_datetime(retrieved_data['Datetime'])

retrieved_data.rename(columns={'Datetime': 'Timestamp'}, inplace=True)

retrieved_data['Day'] = retrieved_data['Timestamp'].dt.strftime('%Y-%m-%d')
day_mapping = {day: idx + 1 for idx, day in enumerate(sorted(retrieved_data['Day'].unique()))}
retrieved_data['Day'] = retrieved_data['Day'].map(day_mapping)
retrieved_data['Day'] = retrieved_data['Day'].astype(int)

# Initialize lists to store ORB, PM, and Yest values
orb_highs = []
orb_lows = []
pm_highs = []
pm_lows = []
yest_highs = []
yest_lows = []

# Loop through each distinct day in the retrieved_data dataframe
for day in retrieved_data['Day'].unique():
    # Create a daily dataframe for the current day
    daily_data = retrieved_data[retrieved_data['Day'] == day]
    
    # Calculate ORB_High and ORB_Low (first 15 minutes of open market)
    orb_data = daily_data[
        (daily_data['Timestamp'].dt.time >= datetime.strptime(market_open, '%H:%M:%S').time()) &
        (daily_data['Timestamp'].dt.time < (datetime.strptime(market_open, '%H:%M:%S') + timedelta(minutes=15)).time())
    ]
    orb_highs.append(orb_data['High'].max())
    orb_lows.append(orb_data['Low'].min())
    
    # Calculate PM_High and PM_Low (pre-market hours)
    pm_data = daily_data[
        (daily_data['Timestamp'].dt.time >= datetime.strptime(pre_market_start, '%H:%M:%S').time()) &
        (daily_data['Timestamp'].dt.time < datetime.strptime(market_open, '%H:%M:%S').time())
    ]
    pm_highs.append(pm_data['High'].max())
    pm_lows.append(pm_data['Low'].min())
    
    # Calculate Yest_High and Yest_Low (previous day's high and low)
    if len(yest_highs) == 0:  # No previous day for the first day
        yest_highs.append(None)
        yest_lows.append(None)
    else:
        prev_day_data = retrieved_data[retrieved_data['Day'] == (day - 1)]
        yest_highs.append(prev_day_data['High'].max())
        yest_lows.append(prev_day_data['Low'].min())

# Map the calculated values back to the retrieved_data dataframe
retrieved_data['ORB_High'] = retrieved_data['Day'].map(dict(zip(retrieved_data['Day'].unique(), orb_highs)))
retrieved_data['ORB_Low'] = retrieved_data['Day'].map(dict(zip(retrieved_data['Day'].unique(), orb_lows)))
retrieved_data['PM_High'] = retrieved_data['Day'].map(dict(zip(retrieved_data['Day'].unique(), pm_highs)))
retrieved_data['PM_Low'] = retrieved_data['Day'].map(dict(zip(retrieved_data['Day'].unique(), pm_lows)))
retrieved_data['Yest_High'] = retrieved_data['Day'].map(dict(zip(retrieved_data['Day'].unique(), yest_highs)))
retrieved_data['Yest_Low'] = retrieved_data['Day'].map(dict(zip(retrieved_data['Day'].unique(), yest_lows)))

retrieved_data['Session'] = retrieved_data['Timestamp'].dt.time.apply(
    lambda t: 'PM' if datetime.strptime(pre_market_start, '%H:%M:%S').time() <= t < datetime.strptime(market_open, '%H:%M:%S').time()
    else 'Regular Market' if datetime.strptime(market_open, '%H:%M:%S').time() <= t < datetime.strptime(market_close, '%H:%M:%S').time()
    else None
)

# Apply Signal Generation
retrieved_data = generate_signals(retrieved_data)

C:\Users\anton\AppData\Local\Temp\ipykernel_14464\2166447052.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\anton\AppData\Local\Temp\ipykernel_14464\2166447052.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\anton\AppData\Local\Temp\ipykernel_14464\2166447052.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

# Candlestick Chart with Indicators

This cell generates an interactive candlestick chart using Plotly, enriched with various technical indicators to provide insights into stock price movements. The chart includes:

- **Candlestick Data**: Visual representation of open, high, low, and close prices for each interval.
- **Indicators**:
    - **ORB High/Low**: Opening Range Breakout levels.
    - **Yesterday's High/Low**: Previous day's high and low prices.
    - **PM High/Low**: Pre-market high and low prices.
    - **8EMA**: Exponential Moving Average with a period of 8.
    - **VWAP**: Volume Weighted Average Price.
- **Session Data**:
    - Pre-market and regular market sessions are distinguished with separate line plots for their respective closing prices.

The chart is interactive, allowing users to hover over data points for detailed information and toggle between different views for better analysis.

In [219]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from generate_candlestick_df import generate_candlestick

# Generate candlestick data
candlestick_data = generate_candlestick(retrieved_data, intervalAmt) # change this to change the interval on the candlestick chart
candlestick_data.set_index('Timestamp', inplace=True)

def plot_candlestick_with_indicators(candlestick_data, simulated_data):
    # Create a figure with a single subplot
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True, 
                        vertical_spacing=0.1, 
                        subplot_titles=("Candlestick Plot",))
    # Add candlestick traces
    fig.add_trace(go.Candlestick(
        x=candlestick_data.index,
        open=candlestick_data['Open'],
        high=candlestick_data['High'],
        low=candlestick_data['Low'],
        close=candlestick_data['Close'],
        name='Candlestick',
        opacity=1
    ), row=1, col=1)

    # Add indicators to the candlestick plot
    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['ORB_High'],
        mode='lines',
        name='ORB High',
        line=dict(color='green', dash='dash')
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['ORB_Low'],
        mode='lines',
        name='ORB Low',
        line=dict(color='red', dash='dash')
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['Yest_High'],
        mode='lines',
        name="Yesterday's High",
        line=dict(color='gray', dash='dashdot')
    ), row=1, col=1)
    
    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['Yest_Low'],
        mode='lines',
        name="Yesterday's Low",
        line=dict(color='brown', dash='dashdot')
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['PM_High'],
        mode='lines',
        name='PM High',
        line=dict(color='green', dash='dot')
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['PM_Low'],
        mode='lines',
        name='PM Low',
        line=dict(color='red', dash='dot')
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['8EMA'],
        mode='lines',
        name='8EMA',
        line=dict(color='orange')
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['VWAP'],
        mode='lines',
        name='VWAP',
        line=dict(color='blue')
    ), row=1, col=1)

    # Add PM and Regular Market values
    fig.add_trace(go.Scatter(
        x=simulated_data[simulated_data['Session'] == 'PM']['Timestamp'],
        y=simulated_data[simulated_data['Session'] == 'PM']['Close'],
        mode='lines',
        name='Pre-Market Value',
        line=dict(color='black', dash='dot'),
        opacity=0.7
    ), row=1, col=1)
    
    fig.add_trace(go.Scatter(
        x=simulated_data[simulated_data['Session'] == 'Regular Market']['Timestamp'],
        y=simulated_data[simulated_data['Session'] == 'Regular Market']['Close'],
        mode='lines',
        name='Regular Market Value',
        line=dict(color='steelblue'),
        opacity=0.8
    ), row=1, col=1)

    # Update layout for better appearance
    fig.update_layout(
        title="Simulated Stock Price with Indicators",
        xaxis_title="Timestamp",
           yaxis_title="Price",
        legend_title="Legend",
        xaxis=dict(tickangle=45),
        template="seaborn",
        hovermode='x unified',
        height=800  # Make the graph taller
    )

    # Show the plot
    fig.show()

# Example usage
plot_candlestick_with_indicators(candlestick_data, retrieved_data)

In [220]:
from getting_options import black_scholes_dataframe
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


def preprocess_intraday_df(df):
    df = df.copy()
    if 'datetime' in df.columns:
        df['datetime'] = pd.to_datetime(df['datetime'])
    return df

def test_strategy_from_df(df,
                          initial_capital,               # Starting cash balance for the strategy
                          risk_per_trade,               # Fraction of portfolio to risk per trade (e.g. 0.1 = 10%)
                          open_range_min,               # Number of minutes after open to define breakout range
                          stop_loss_pct,                # Percentage loss that triggers a stop on a trade
                          take_profit_pct,              # Percentage gain that triggers profit-taking
                          expiration_days,              # Days to expiration used in Black-Scholes option pricing
                          sigma,                        # Assumed/implied volatility used in Black-Scholes model
                          max_trades=3,                 # Max number of trades allowed per day
                          max_minutes_in_trade=60,      # Max time (in minutes) a position can be held
                          max_daily_loss_pct=0.15,      # Max % of portfolio you can lose in a single day before halting trades
                          max_portfolio_cap=4,          # Cap portfolio growth (e.g. 4x starting capital) to avoid unrealistic compounding
                          max_drawdown_pct=0.70,        # Max intraday drawdown allowed (reset daily)
                          max_consecutive_losses=7,     # Number of losing trades in a row before trading halts for the day
                          trailing_stop_pct=0.25,       # Trailing stop to protect profits if price falls from recent high
                          decay_threshold=0.03):        # If no price movement after max_minutes, this % threshold triggers an exit

    df = preprocess_intraday_df(df)

    if df is None or df.empty:
        print("\u26a0\ufe0f No data provided.")
        return None

    df = df.copy()
    df = identify_breakout_levels(df, open_range_min=open_range_min)
    df = determine_trade_direction(df)
    df = black_scholes_dataframe(df, sigma=sigma, expiration_days=expiration_days)

    df.set_index('datetime', inplace=True)
    portfolio = float(initial_capital)
    position = None
    entry_price = None
    entry_time = None
    trade_size = 0.0
    contracts = 0
    total_cost = 0.0
    df['Portfolio Value'] = float(portfolio)
    df['Realized PnL'] = 0.0

    df['range'] = df['high'] - df['low']
    df['avg_range'] = df['range'].rolling(15).mean()
    volatility_threshold = df['avg_range'].quantile(0.5)
    df['Vol_Filter'] = df['avg_range'] > volatility_threshold

    trade_log = []
    current_day = None
    trade_count = 0
    consecutive_losses = 0
    daily_portfolio_high = portfolio

    for i in range(1, len(df)):
        now = df.index[i]
        trade_time = now.time()
        if trade_time < pd.to_datetime("09:30:00").time() or trade_time > pd.to_datetime("16:00:00").time():
            continue

        new_day = now.date()
        if new_day != current_day:
            current_day = new_day
            trade_count = 0
            consecutive_losses = 0
            daily_portfolio_high = portfolio

        day_trades = [log for log in trade_log if pd.to_datetime(log['Exit Time']).date() == current_day]
        daily_pnl = sum([log['PnL'] for log in day_trades])
        if daily_pnl < -max_daily_loss_pct * portfolio:
            continue

        # Daily drawdown based on daily portfolio high
        daily_drawdown = (portfolio / daily_portfolio_high) - 1
        if daily_drawdown < -max_drawdown_pct:
            continue

        if consecutive_losses >= max_consecutive_losses:
            continue

        if trade_count >= max_trades:
            continue

        if not df.iloc[i]['Vol_Filter']:
            continue

        trade_type = df.iloc[i]['Trade_Type']

        if trade_type in ["CALL", "PUT"] and position is None:
            entry_price = df.iloc[i]['Call_Price'] if trade_type == "CALL" else df.iloc[i]['Put_Price']
            if np.isnan(entry_price) or entry_price <= 0 or entry_price > 100:
                continue

            drawdown_ratio = (portfolio / daily_portfolio_high) - 1
            adjusted_risk = max(0.01, risk_per_trade * (1 + drawdown_ratio))
            trade_size = adjusted_risk * portfolio

            contracts = int(trade_size // (entry_price * 100))
            if contracts == 0:
                continue

            position = trade_type
            entry_time = now
            total_cost = contracts * entry_price * 100
            peak_price = entry_price

        elif position is not None:
            is_eod = (i + 1 == len(df)) or (df.index[i + 1].date() != current_day)

            current_price = df.iloc[i]['Call_Price'] if position == "CALL" else df.iloc[i]['Put_Price']
            if np.isnan(current_price):
                continue

            peak_price = max(peak_price, current_price)
            trailing_stop_triggered = current_price <= peak_price * (1 - trailing_stop_pct)

            stop_loss_level = entry_price * (1 - stop_loss_pct)
            take_profit_level = entry_price * (1 + take_profit_pct)
            time_in_trade = (now - entry_time).total_seconds() / 60 if entry_time else 0
            decay_exit = time_in_trade >= max_minutes_in_trade and abs(current_price - entry_price) / entry_price < decay_threshold

            exit_condition = (
                current_price <= stop_loss_level or
                current_price >= take_profit_level or
                trailing_stop_triggered or
                is_eod or
                decay_exit
            )

            if exit_condition:
                if current_price <= stop_loss_level:
                    reason = 'Stop Loss'
                elif current_price >= take_profit_level:
                    reason = 'Take Profit'
                elif trailing_stop_triggered:
                    reason = 'Trailing Stop'
                elif is_eod:
                    reason = 'EOD Force Exit'
                else:
                    reason = 'Time/Price Decay'

                trade_profit = contracts * (current_price - entry_price) * 100
                if abs(trade_profit) > 0.5 * portfolio:
                    print(f"Large trade: {trade_profit:.2f} at {now}")

                portfolio += trade_profit
                daily_portfolio_high = max(daily_portfolio_high, portfolio)
                df.iloc[i, df.columns.get_loc('Realized PnL')] = float(trade_profit)

                trade_log.append({
                    'Entry Time': entry_time,
                    'Exit Time': now,
                    'Type': position,
                    'Entry Price': entry_price,
                    'Exit Price': current_price,
                    'PnL': trade_profit,
                    'Contracts': contracts,
                    'Total Cost': total_cost,
                    'Reason': reason
                })

                position = None
                entry_price = None
                trade_size = 0.0
                contracts = 0
                total_cost = 0.0
                entry_time = None
                trade_count += 1

                if trade_profit < 0:
                    consecutive_losses += 1
                else:
                    consecutive_losses = 0

        if portfolio <= 0:
            print("Portfolio wiped out")
            break

        portfolio = min(portfolio, max_portfolio_cap * initial_capital)
        df.iloc[i, df.columns.get_loc('Portfolio Value')] = float(portfolio)

    df['Market Return'] = df['close'].pct_change()
    df['Cumulative Market Return'] = (1 + df['Market Return']).cumprod()
    df['Cumulative Strategy Return'] = df['Portfolio Value'] / initial_capital

    realized = df['Realized PnL']
    valid_pnls = realized[realized != 0]
    win_rate = (valid_pnls > 0).mean() if not valid_pnls.empty else 0.0

    sharpe_ratio = realized.mean() / realized.std() * np.sqrt(252) if realized.std() > 0 else 0
    rolling_max = df['Cumulative Strategy Return'].cummax()
    drawdown = (df['Cumulative Strategy Return'] / rolling_max) - 1
    max_drawdown = drawdown.min()

    reason_summary = pd.DataFrame(trade_log).groupby("Reason")['PnL'].agg(['count', 'mean', 'sum'])
    print("\nExit Reason Summary:\n", reason_summary)

    plt.figure(figsize=(12, 6))
    plt.plot(df.index, df['Cumulative Market Return'], label="Market Return", linestyle="dashed")
    plt.plot(df.index, df['Cumulative Strategy Return'], label="Strategy Return", color='green')
    plt.legend()
    date_range = f"{df.index[0].date()} to {df.index[-1].date()}"
    plt.title(f"Strategy Performance ({date_range})\nSharpe: {sharpe_ratio:.2f}, Max DD: {max_drawdown:.2%}")
    plt.show()

    plt.figure(figsize=(8, 4))
    pd.Series([t['PnL'] for t in trade_log]).hist(bins=100)
    plt.title("PnL Distribution")
    plt.xlabel("Profit/Loss per Trade")
    plt.ylabel("Frequency")
    plt.show()

    total_trades = len(trade_log)

    return {
        'Sharpe Ratio': round(sharpe_ratio, 2),
        'Max Drawdown': round(max_drawdown * 100, 2),
        'Win Rate': round(win_rate * 100, 2),
        'Final Portfolio Value': round(portfolio, 2),
        'Final Strategy Return': round(df['Cumulative Strategy Return'].iloc[-1] * 100, 2),
        'Total Trades': total_trades,
        'DataFrame': df.reset_index(),
        'Trade Log': pd.DataFrame(trade_log)
    }


# identify_breakout_levels and determine_trade_direction stay the same

def determine_trade_direction(df):
    df = df.copy()
    df['Breakout_Long'] = df['high'] > df['OpenRange_High']
    df['Breakout_Short'] = df['close'] < df['OpenRange_Low']

    df['Confirmed_Breakout'] = df['Breakout_Long']
    df['Breakout_Failure'] = df['Breakout_Long'] & (df['close'] < df['OpenRange_High'])
    df['Breakout_Failure_Short'] = df['Breakout_Failure'] & (df['close'] < df['OpenRange_Low'])

    df['Trade_Type'] = 'NONE'
    df.loc[df['Confirmed_Breakout'], 'Trade_Type'] = 'CALL'
    df.loc[df['Breakout_Failure_Short'], 'Trade_Type'] = 'PUT'

    return df


def identify_breakout_levels(df, open_range_min=15):
    df = df.copy()
    df['date'] = df['datetime'].dt.date

    breakout_levels = []
    for day, group in df.groupby('date'):
        market_open = pd.to_datetime(f"{day} 09:30:00-04:00")
        open_range_end = market_open + pd.Timedelta(minutes=open_range_min)

        open_range = group[(group['datetime'] >= market_open) & (group['datetime'] <= open_range_end)]
        pre_market = group[group['datetime'] < market_open]
        prev_day = df[df['date'] == (pd.to_datetime(day) - pd.Timedelta(days=1)).date()]

        high = open_range['high'].max() if not open_range.empty else np.nan
        low = open_range['low'].min() if not open_range.empty else np.nan
        pm_high = pre_market['high'].max() if not pre_market.empty else np.nan
        pm_low = pre_market['low'].min() if not pre_market.empty else np.nan
        prev_high = prev_day['high'].max() if not prev_day.empty else np.nan
        prev_low = prev_day['low'].min() if not prev_day.empty else np.nan

        group = group.copy()
        group['OpenRange_High'] = high
        group['OpenRange_Low'] = low
        group['PreMarket_High'] = pm_high
        group['PreMarket_Low'] = pm_low
        group['PrevDay_High'] = prev_high
        group['PrevDay_Low'] = prev_low

        breakout_levels.append(group)

    return pd.concat(breakout_levels)

ImportError: cannot import name 'RESTClient' from 'polygon' (c:\Users\anton\AppData\Local\Programs\Python\Python313\Lib\site-packages\polygon\__init__.py)

In [ ]:
df = pd.read_csv('df_2022_2024.csv', parse_dates=['datetime'])
df['datetime'] = pd.to_datetime(df['datetime'], utc=True).dt.tz_convert('America/New_York')
df


results = test_strategy_from_df(
    df,
    initial_capital=20000,
    risk_per_trade=0.20,
    open_range_min=15,
    stop_loss_pct=0.10,
    take_profit_pct=0.20,
    expiration_days=1,
    sigma=0.20,
    max_trades=1
)

print("Backtest Results:")
for k, v in results.items():
    if k not in ['DataFrame', 'Trade Log']:  
        print(f"{k}: {v}")
